In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType
from pyspark.sql.window import Window
from pyspark.sql.functions import col, trim, current_time

In [0]:
df = spark.read.table("dev_project.bronze.crm_cust_info")